In [1]:
import argparse
import numpy as np
import os
import pandas as pd
import statistics as stat

# import pymc3 as pm
import pickle
import matplotlib.pyplot as plt
from google.protobuf import text_format
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm, mode
from scipy.interpolate import griddata
from sklearn.metrics import adjusted_rand_score
from math import sqrt
from itertools import product

import sys
sys.path.append('.')
sys.path.append('./pp_mix')
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix


In [3]:
p_s = [100,200]
d_s = [2,5]
M_s = [4,8]
n_percluster_s =[50] 

for p,dtrue,M,npc in product(p_s, d_s, M_s, n_percluster_s):
    
    outpath_d = "data/Student_data/applam/app_p_{0}_d_{1}_M_{2}_npc_{3}_out".format(p,dtrue,M,npc)
    
    rho_s = [5, 10, 20]

    for rho in rho_s:
        
        base_outpath_rho = os.path.join(outpath_d, "rho_{0}_out".format(rho)) + "_{0}"
        i = 0
        while not(os.path.exists(base_outpath_rho.format(i))):
            i = i+1
        
        outpath = base_outpath_rho.format(i)
        
        chain = loadChains(os.path.join(outpath, "chains.recordio") , MultivariateMixtureState)
        
         # plots
        fig = plt.figure()
        tau_chain = np.array([x.lamb_block.tau for x in chain])
        plt.plot(tau_chain)
        plt.title("tau chain")
        plt.savefig(os.path.join(outpath, "tau_chain.pdf"))
        plt.close()

        fig = plt.figure()
        first_sbar_chain = np.array([to_numpy(x.sigma_bar)[0] for x in chain])
        plt.plot(first_sbar_chain,color='red')
        last_sbar_chain = np.array([to_numpy(x.sigma_bar)[-1] for x in chain])
        plt.plot(last_sbar_chain,color='blue')
        plt.title("sbar_chain")
        plt.savefig(os.path.join(outpath, "sbar_chain.pdf"))
        plt.close()

        # Compute Posterior Summaries
        fig = plt.figure()
        n_cluster_chain = np.array([x.ma for x in chain])
        plt.plot(n_cluster_chain)
        plt.title("number of clusters chain")
        plt.savefig(os.path.join(outpath, "nclus_chain.pdf"))
        plt.close()

        fig = plt.figure()
        n_nonall_chain = np.array([x.mna for x in chain])
        plt.plot(n_nonall_chain)
        plt.title("number of non allocated components chain")
        plt.savefig(os.path.join(outpath, "non_alloc_chain.pdf"))
        plt.close()


